In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Review of Decision Trees - yet another classifier model

**Decision trees can be described by the following:**
- Considered to be one of the most mature and traditional algorithms in predictive analytics
- Used to solve classification problems through visual and explicit representations of decisions and decision making
- Essentially a map where you can follow the path according to your decisions at every step, and at the end you get your prediction

## Why and when do we need Decision Trees?

- When features are catgorical: when we can classify data into known groups
- When we want to model a set of sequential, heirarchical decisions that lead to some final result
- When we need to explain (to a boss) the reasoning behind a specific decision

## What are the components of Decision Trees?
**Here is what makes up a decision tree:**
- Root Node
- Leaf Node

We can obtain these root and leaf nodes via:
- Conditional probability
- Entropy
- Information Gain

# The steps and defenitions behind building a Decision Tree

## Entropy
Entropy is a measure of uncertainty for a random variable. The higher the entropy, the more uncertain we are of the variable value at a certain event

$H(Coin) = \sum -p(outcome) * log_2(p(outcome)$

Where the coin has two outcomes, for example: p(H) = .5 and p(T) = .5

In [2]:
def entropy(p):
    H = np.array([-i*np.log2(i) for i in p]).sum()
    return H
    
p = [.5, .5]
print(entropy(p))

1.0


The entropy of the coin with probability of .5 has the most uncertainty when we flip it, so the entropy is at the max of 1.0

In [3]:
p = [.9, .1]
print(entropy(p))

0.4689955935892812


Here we can see that an unfair coin has a way lower entropy, meaning we are more certain of the outcome

## Conditional Probability
Conditional probability can be described as the answer to this question:  
>Given that some friends like Chocolate, what is the probability that they like Strawberry as well?

The formula that we can use to find that probability is this one:
  
  
$ P( Strawberry \mid Chocolate ) = \frac{P( Chocolate \cap Strawberry )}{P( Chocolate )} $

In [20]:
df = pd.read_csv('tennis.csv', delimiter='\t', names=['1', 'Outlook', 'Temp', 'Humidity', 'Wind', 'Decision_to_play'])
df = df.drop(['1'], axis=1)

In [22]:
def obtain_conditional_prob(input_df, col, condition, target):
    """
    Obtain conditional probability of decision being yes or no given a df column and a condition
    Return two probabilities in a tuple
    """
    
    df = input_df
    if condition != 'Target':
        df = input_df[input_df[col] == condition]
    df = df[[col, target]]
    
    total = len(df)
    pos = df[target].value_counts()[0]
    neg = total - pos
    
    return (pos/total, neg/total)

# what are the probabilities of Decision_to_play given Wind is Weak?
print(obtain_conditional_prob(df,'Wind', 'Weak', 'Decision_to_play'))

# what are the probabilities of Decision_to_play given Wind is Strong?
print(obtain_conditional_prob(df, 'Wind', 'Strong', 'Decision_to_play'))

(0.75, 0.25)
(0.5, 0.5)


## Information Gain

Information gain measures how much information a feature gives us about the decision (output class)  
> This is the main feature used by a Decision Tree algorithm to construct a Decision Tree

Decision Trees will always try to maximize information gain and the higher the information gain that a feature has, the more likely it is to be at the top of the tree

**Here is the formula for calculating Information Gain for a specific feature:**  
  
$I(Decision; Wind) = H(Decision) - \sum P(Wind) * H(Decision | Wind)$

In [23]:
def obtain_feature_prob(input_df, col, condition):
    """
    Return probability of a certain condition being true in a single column(feature)
    """
    total = len(input_df[col])
    val = len(input_df[input_df[col] == condition])
    return val/total

In [24]:
def get_entropy(p_list):
    """ 
    Returns entropy for a tuple of probabilities. 
    This represents uncertainty of the tuple as a whole. The bigger the entropy, the more uncertainty
    Formula = -[P(H)*log^2(H) + P(T)*log^2(T)]
    """
    num1 = p_list[0]
    num2 = p_list[1]
    
    # Edge Case
    if num1 == 0 or num2 == 0:
        return 0

    return -(num1*np.log2(num1) + num2*np.log2(num2))

In [25]:
def get_info_gain(df, col, decision):
    """
    Calculate and return the info gain for a specific feature
    """
    
    conditions = df[col].value_counts().keys()
    
    # Obtain the entropy of the decision column itself
    H_decision = get_entropy(obtain_conditional_prob(df, col, 'Target', decision))
    
    # Calculate the summation part of the formula
    p_s = []
    for condition in conditions:
        val = obtain_feature_prob(df, col, condition) * get_entropy(obtain_conditional_prob(df, col, condition, decision))
        p_s.append(val)
        
    return H_decision - (np.sum(p_s))

In [26]:
for col in df.columns[:-1]:
    info_gain = get_info_gain(df, col, df.columns[-1:][0])
    print(f'info gain between {col} and Decision is {info_gain}')

info gain between Outlook and Decision is 0.24674981977443933
info gain between Temp and Decision is 0.02922256565895487
info gain between Humidity and Decision is 0.15183550136234159
info gain between Wind and Decision is 0.04812703040826949


### So we can see that the Info Gain for the Outlook feature was the highest, so we should use it as the root of the Decision Tree

## We can now branch out from the Outlook feature, and build sub-decision trees
We stop when we reach either a set depth, or 